# Persistent Volumes (PV) & PersistentVolumeClaims (PVC)

A **PVC** is a request for persistent storage. It lets pods use durable storage that survives pod restarts/rescheduling.

Related objects:
- **PV**: the actual piece of storage (often provisioned dynamically).
- **StorageClass**: describes *how* to provision storage (cloud disk, CSI driver settings).


## Why it is used
- Pods are ephemeral; local container filesystems are not durable.
- Stateful apps (databases, queues) need persistent disk.
- Decouple "who uses storage" (PVC) from "how storage is provided" (StorageClass/PV).


## YAML template: PVC (pseudo)
```yaml
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: app-data
spec:
  accessModes:
    - ReadWriteOnce
  storageClassName: standard
  resources:
    requests:
      storage: 10Gi
```


## How a pod uses a PVC
```yaml
volumes:
  - name: data
    persistentVolumeClaim:
      claimName: app-data
containers:
  - name: app
    volumeMounts:
      - name: data
        mountPath: /var/lib/app
```


## Key concepts
- **Access modes**:
  - `ReadWriteOnce` (RWO): mounted read-write by a single node.
  - `ReadWriteMany` (RWX): mounted by many nodes (depends on storage).
- **Dynamic provisioning**: PVC triggers PV creation using a StorageClass.
- **Default StorageClass**: if `storageClassName` is omitted, Kubernetes may use the cluster default (which can differ between clusters).
- **Reclaim policy**: what happens to PV after PVC is deleted (Delete/Retain).


## Pitfalls
- RWX is not supported by all storage backends; pick the right storage class.
- Relying on an implicit default StorageClass is a portability trap; define one explicitly and reference it.
- Backups are not automatic: plan snapshots/backups outside Kubernetes too.
- Resizing requires support in the storage class and filesystem.

## References
- Volumes: https://kubernetes.io/docs/concepts/storage/volumes/
- Persistent Volumes: https://kubernetes.io/docs/concepts/storage/persistent-volumes/
